In [ ]:
一、使用pytorch搭建神经网络模型，实现对KMNIST数据集的训练。

https://pytorch.org/vision/stable/generated/torchvision.datasets.KMNIST.html#torchvision.datasets.KMNIST



In [1]:
# 导入相应库
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 1.定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(), #将图像转化为张量
    transforms.Normalize((0.1307),(0.3081,)) # 归一化处理
    ])

In [12]:
# 2.加载KMNIST数据及
train_dataset = datasets.KMNIST(root='./data',train=True,
                                download=True,transform=transform)

test_dataset = datasets.KMNIST(root='./data', train=False,
                            download=True, transform=transform)


In [13]:
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# 3. 定义神经网络
class KMNISTNet(nn.Module):
    def __init__(self):
        super(KMNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, training=self.training)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

model = KMNISTNet()

In [7]:
# 4.定义损失函数和优化器
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [8]:
# 5.训练模型
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
# 6.评估模型
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [14]:
# 训练模型多个 epoch
epochs = 5
for epoch in range(1, epochs + 1):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318684
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.123424
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.684642
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.459090
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.559723
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.356051
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.085390
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.089654
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.993684
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.038062

Test set: Average loss: 0.0165, Accuracy: 6666/10000 (67%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.059397
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.919843
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.932759
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.040036
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.762416
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.950219
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.011353
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.842264
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.915004
T

In [17]:
# 二、尝试调整模型结构（变更神经元数量，增加隐藏层）来提升模型预测的准确率

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.1307,), (0.3081,))  # 归一化处理
])

# 2. 加载 KMNIST 数据集
train_dataset = datasets.KMNIST(root='./data', train=True,
                                download=True, transform=transform)
test_dataset = datasets.KMNIST(root='./data', train=False,
                               download=True, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 3. 定义改进后的神经网络模型
class ImprovedKMNISTNet(nn.Module):
    def __init__(self):
        super(ImprovedKMNISTNet, self).__init__()
        # 第一个卷积层，增加输出通道数
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        # 第二个卷积层
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        # 第三个卷积层
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        # 增加全连接层的神经元数量并添加额外的隐藏层
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # 第一个卷积块
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        # 第二个卷积块
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        # 第三个卷积块
        x = self.pool(nn.functional.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 3 * 3)
        # 第一个全连接层
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        # 第二个全连接层
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        # 输出层
        x = self.fc3(x)
        return nn.functional.log_softmax(x, dim=1)

model = ImprovedKMNISTNet()

# 4. 定义损失函数和优化器
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 训练模型
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# 6. 评估模型
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# 训练模型多个 epoch
epochs = 10
for epoch in range(1, epochs + 1):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306040
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.573660
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.329039
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.231981
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.060837
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.133807
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.093759
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.027547
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.195032
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.141461

Test set: Average loss: 0.0038, Accuracy: 9271/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.050481
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.066839
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.036027
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.040633
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.148482
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.017655
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.054590
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.028459
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.025328
T

In [ ]:
# 3. 调试超参数，观察学习率和批次大小对训练的影响。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载 KMNIST 数据集
train_dataset = datasets.KMNIST(root='./data', train=True,
                                download=True, transform=transform)
test_dataset = datasets.KMNIST(root='./data', train=False,
                               download=True, transform=transform)

# 定义神经网络模型
class KMNISTNet(nn.Module):
    def __init__(self):
        super(KMNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# 训练模型
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 评估模型
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

# 不同的学习率和批次大小组合
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]

# 训练不同超参数组合
for lr in learning_rates:
    for batch_size in batch_sizes:
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        model = KMNISTNet()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        epochs = 10
        for epoch in range(1, epochs + 1):
            train(model, train_loader, criterion, optimizer, epoch)

        accuracy = test(model, test_loader, criterion)
        print(f'Learning Rate: {lr}, Batch Size: {batch_size}, Test Accuracy: {accuracy:.2f}%')